In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [9]:

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2


In [10]:

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

In [11]:
# radiohead lyrics from Manik2000
text = pd.read_csv("https://raw.githubusercontent.com/Manik2000/radiohead-lyrics/main/data/lyrics.csv").query("title=='Fake Plastic Trees'")['lyrics'].iloc[0]
# cut out random notes at the start of the field and at the end
text = text.split("Lyrics\n")[1].split("88Embed")[0]
text

"A green plastic watering can\nFor a fake Chinese rubber plant\nIn a fake plastic earth\nThat she bought from a rubber man\nIn a town full of rubber plants\nTo get rid of itself\n\nIt wears her out\nIt wears her out\nIt wears her out\nIt wears her out\n\nShe lives with a broken man\nA cracked polystyrene man\nWho just crumbles and burns\nHe used to do surgery\nFor girls in the eighties\nBut gravity always wins\n\nAnd it wears him out\nIt wears him out\nIt wears him out\nIt wears\nYou might also like\nShe looks like the real thing\nShe tastes like the real thing\nMy fake plastic love\nBut I can't help the feeling\nI could blow through the ceiling\nIf I just turn and run\n\nAnd it wears me out\nIt wears me out\nIt wears me out\nIt wears me out\n\nAnd if I could be who you wanted\nIf I could be who you wanted\nAll the time\nAll the time"

In [12]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  805


# Encoding of characters

This is very simple. But there are other methods like `tiktoken`, `sentencepiece` etc.

In [13]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocab size: {vocab_size}")
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }  # string to integer
itos = { i:ch for i,ch in enumerate(chars) }  # integer to string
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


vocab size: 37


In [14]:
print(encode("fake plastic trees"))
print(decode(encode("fake plastic trees")))

[19, 14, 24, 18, 1, 29, 25, 14, 31, 32, 22, 16, 1, 32, 30, 18, 18, 31]
fake plastic trees


In [15]:
stoi

{'\n': 0,
 ' ': 1,
 "'": 2,
 'A': 3,
 'B': 4,
 'C': 5,
 'F': 6,
 'H': 7,
 'I': 8,
 'M': 9,
 'S': 10,
 'T': 11,
 'W': 12,
 'Y': 13,
 'a': 14,
 'b': 15,
 'c': 16,
 'd': 17,
 'e': 18,
 'f': 19,
 'g': 20,
 'h': 21,
 'i': 22,
 'j': 23,
 'k': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'y': 36}

In [16]:
# Encode and store it in a torch.tensor object
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long) # torch.long is desired memory format of Tensor
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
# `data` is just numbers representing the lyrics
decode(encode(text))

"A green plastic watering can\nFor a fake Chinese rubber plant\nIn a fake plastic earth\nThat she bought from a rubber man\nIn a town full of rubber plants\nTo get rid of itself\n\nIt wears her out\nIt wears her out\nIt wears her out\nIt wears her out\n\nShe lives with a broken man\nA cracked polystyrene man\nWho just crumbles and burns\nHe used to do surgery\nFor girls in the eighties\nBut gravity always wins\n\nAnd it wears him out\nIt wears him out\nIt wears him out\nIt wears\nYou might also like\nShe looks like the real thing\nShe tastes like the real thing\nMy fake plastic love\nBut I can't help the feeling\nI could blow through the ceiling\nIf I just turn and run\n\nAnd it wears me out\nIt wears me out\nIt wears me out\nIt wears me out\n\nAnd if I could be who you wanted\nIf I could be who you wanted\nAll the time\nAll the time"

# Train the model

We don't feed the whole training data into the model. Too computationally expensive. Instead, we feed random chunks of data at a time. The size of these chunks is called  `block_size` in this example but others may call it `context_length`.

In [18]:
block_size = 8
train_data[:block_size+1]

tensor([ 3,  1, 20, 30, 18, 18, 27,  1, 29])

In this particular example, there's actually a lot of examples packed into it because of the masking.

`3`
<br>
`3,  1`
<br>
`3,  1, 20`  # in the context of seeing `3, 1` then `20` comes next
<br>
etc.

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is: {context} the target is: {target}")

when input is: tensor([3]) the target is: 1
when input is: tensor([3, 1]) the target is: 20
when input is: tensor([ 3,  1, 20]) the target is: 30
when input is: tensor([ 3,  1, 20, 30]) the target is: 18
when input is: tensor([ 3,  1, 20, 30, 18]) the target is: 18
when input is: tensor([ 3,  1, 20, 30, 18, 18]) the target is: 27
when input is: tensor([ 3,  1, 20, 30, 18, 18, 27]) the target is: 1
when input is: tensor([ 3,  1, 20, 30, 18, 18, 27,  1]) the target is: 29


This also helps the transformer get used to seeing contexts at these size ranges (from 1 to `block_size`). This is also helpful during inference so the model knows what sizes it can predict. Keep in mind that this also limits what the model can predict. After `block_size` the model has to truncate. The above is implicitly the time dimension, since a sequence where order matters is generated.

Now consider the **batch** dimension. It's taking multiple random chunks of text and grouping them together to keep things efficient and keep the GPUs busy since they can be processed in parallel. The chunks are processed completely independently.

In [30]:
torch.manual_seed(1337)  # won't match video's since I'm using a different dataset
block_size = 8   # the maximum context length for predictions
batch_size = 4   # how many independent sequences will we process in parallel


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y;
    # this will happen on any forward or backward pass through the model
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # get random off sets
    x = torch.stack([data[i:i+block_size] for i in ix])        # then stack them all up together
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])    # needed to generate the loss function
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print('\ntargets:')
print(yb.shape)
# the number of rows is the number of batches
# the number of columns is the block_size
print('\nyb:\n', yb)


inputs:
torch.Size([4, 8])

targets:
torch.Size([4, 8])

yb:
 tensor([[33, 20, 21, 32,  1, 19, 30, 28],
        [18,  1,  5, 21, 22, 27, 18, 31],
        [ 1, 26, 18,  1, 28, 33, 32,  0],
        [31,  1, 21, 18, 30,  1, 28, 33]])


In [31]:
xb # our input to the transformer

tensor([[28, 33, 20, 21, 32,  1, 19, 30],
        [24, 18,  1,  5, 21, 22, 27, 18],
        [31,  1, 26, 18,  1, 28, 33, 32],
        [30, 31,  1, 21, 18, 30,  1, 28]])

In [32]:
# again, each is a random chunk of data or lyrics
# represented by integer values. you can verify this here
# this is just the first batch of `yb`
decode(np.array(yb)[0])

'ught fro'

In [33]:
for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()}, the target is: {target}")

when input is [28], the target is: 33
when input is [28, 33], the target is: 20
when input is [28, 33, 20], the target is: 21
when input is [28, 33, 20, 21], the target is: 32
when input is [28, 33, 20, 21, 32], the target is: 1
when input is [28, 33, 20, 21, 32, 1], the target is: 19
when input is [28, 33, 20, 21, 32, 1, 19], the target is: 30
when input is [28, 33, 20, 21, 32, 1, 19, 30], the target is: 28
when input is [24], the target is: 18
when input is [24, 18], the target is: 1
when input is [24, 18, 1], the target is: 5
when input is [24, 18, 1, 5], the target is: 21
when input is [24, 18, 1, 5, 21], the target is: 22
when input is [24, 18, 1, 5, 21, 22], the target is: 27
when input is [24, 18, 1, 5, 21, 22, 27], the target is: 18
when input is [24, 18, 1, 5, 21, 22, 27, 18], the target is: 31
when input is [31], the target is: 1
when input is [31, 1], the target is: 26
when input is [31, 1, 26], the target is: 18
when input is [31, 1, 26, 18], the target is: 1
when input is 

In [34]:
print(xb)

tensor([[28, 33, 20, 21, 32,  1, 19, 30],
        [24, 18,  1,  5, 21, 22, 27, 18],
        [31,  1, 26, 18,  1, 28, 33, 32],
        [30, 31,  1, 21, 18, 30,  1, 28]])


# Feed this into a simple language model

Bigram language model is covered in his make more series.

In [60]:
# super simple bigram model
# nn module is a basic blocking block for graphs 
# https://pytorch.org/docs/stable/nn.html
class BigramLanguageModel(nn.Module):   # subclassing nn.Module

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # for example token 30 (tensor above) will grab the 30th row of this
        # embedding table, see output below
        

    def forward(self, idx, targets=None):
        # idx is x; inputs renamed to idx (not sure why it's renamed)
        # targets is y;

        # idx and targets are both (B,T) tensor of integers
        # logits are the predictions
        logits = self.token_embedding_table(idx) # (B,T,C)  # what is channel? the character size

        # use this conditional to evaluate the loss (quality of predictions)
        # if it is possible
        # likelihood maximization = minimization of neg log likelihood = minimization of the cross-entropy
        # https://discuss.pytorch.org/t/difference-between-cross-entropy-loss-or-log-likelihood-loss/38816/2
        if targets is None:  # this is needed for the `generate` function below where loss isn't used
            loss = None
        else:
            # need to change order of dimensions so it can be read by `cross_entropy`
            B, T, C = logits.shape
            logits = logits.view(B*T, C)  # a 2D view
            targets = targets.view(B*T)  # this will be 1D
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # generate text, by just adding one more for every batch dimension in the time dimension
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)  # loss is being ignored here
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the multinomial probability distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence... whatever is predicted is concatenated
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [61]:
# initialize the model; all we have is embedding
# per the docs  https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
# A simple lookup table that stores embeddings of a fixed dictionary and size.
# it's a thin wrapper around a tensor of vocab_size x vocab_size
m = BigramLanguageModel(vocab_size)
m.token_embedding_table

Embedding(37, 37)

In [62]:
# understanding this embedding table object
# for example token 30 (tensor above) will grab the 30th row of this
# embedding table
# Single index lookup
index = torch.tensor([30])
embedding_vals_idx30 = m.token_embedding_table(index)
print(embedding_vals_idx30.shape)  # Output: torch.Size([1, 300])

torch.Size([1, 37])


In [63]:
embedding_vals_idx30

tensor([[-0.9883,  0.5120,  0.8213, -0.4068,  1.1888, -1.4860,  0.8820,  0.1223,
          0.4179, -0.6261, -0.7949,  0.8804, -0.6083,  0.4067,  0.1444,  0.7383,
          0.2105,  0.7643,  0.6467,  0.6577,  0.3708, -0.4748,  0.5696,  0.8532,
          2.7071,  0.7921, -0.3820, -0.5626,  1.5889,  0.7005, -0.6309, -0.5384,
          2.2132, -0.6222, -0.6999, -0.2165,  2.0037]],
       grad_fn=<EmbeddingBackward0>)

In [64]:
# out = m(xb, yb)
# print(out.shape)

In [65]:
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 37])
tensor(4.1957, grad_fn=<NllLossBackward0>)


In [66]:
# test generation
# idx is being set to a 1x1 tensor of 0
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


u
aYkgBnnA
woMAlgevAWlHbyfvA aojhy
CBaA mvdpSIClBsSts'HiWMTuBevBhncl'HiCyMAkbl'iWjMyWcoHpwBIdgkWMkcv


This is garbage because the model isn't trained.

Also, the bigram model isn't using any history, other than the last character.